# Modelling to Predict Subzone using MLE and Mobile Web GPS as Ground Truth

### Problem Statement

This analysis seeks to predict subzone of users using LBS Data and mobile web GPS data. 
The mobile web GPS data is taken as "ground truth".
* Here we focus on using Random Forest (RF) to predict actual location of the user.

### Data

Data sources:
 
* After oscillation removal and map matched webgps cleaning:
 
/user/terencelzw/dataset_final_v7_mapmatch/20161116

hadoop fs -getmerge /user/terencelzw/dataset_final_v7_mapmatch/20161116 20161116_final_v7_mapmatch.txt

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

from ggplot import *
from multiprocessing import Pool
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import json
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import plot_utilities as pu
import time
import utilities as ut

matplotlib.style.use('ggplot')

#### Input params #################
filename_ROI_in = "./data/sub-zone.json"
# filename_ROI_adjacents_out = "./data/trellis_subzone.pickle"
# filename_ROI_adjacents_csv_out = "./data/trellis_subzone.csv"
filename_ROI_centroidLatLon_out = "./data/ROI_centroidLatLon_subzone.pickle"

filename_data_train_mapmatch = "./data/20161116_20161125_final_v7_mapmatch.txt"
randomState = 10
timeDiffMin = 0 # The min. time diff between mobile and lbs timestamp that we are considering
timeDiffMax = 120 # The max. time diff between mobile and lbs timestamp that we are considering

# filename_data_out = "./data/dataset_20161126_20161128_MLE_v7" + str(randomState) + ".txt"
###################################

columns = ['imsi','cellid','lbs_timestamp','event_type','network','antenna','lbs_lat','lbs_lng',
'radius','azimuth','mean_2_lat','mean_2_lng','mean_4_lat','mean_4_lng','mean_6_lat',
'mean_6_lng','mean_8_lat','mean_8_lng','mean_10_lat','mean_10_lng','prev_cell_id',
'prev_lat', 'prev_lng', 'prev_timestamp','mean_2_lat_f','mean_2_lng_f','mean_4_lat_f','mean_4_lng_f',
'mean_6_lat_f','mean_6_lng_f','mean_8_lat_f','mean_8_lng_f','mean_10_lat_f','mean_10_lng_f',
'future_cell_id','future_lat','future_lng', 'future_timestamp', 'mobile_timestamp','host','url','useragent',
'mobile_lat','mobile_lng','mobile_zone','lbs_zone','tf_lbs_lat','tf_lbs_lng','tf_mean_2_lat',
'tf_mean_2_lng','tf_mean_4_lat','tf_mean_4_lng','tf_mean_6_lat','tf_mean_6_lng','tf_mean_8_lat',
'tf_mean_8_lng','tf_mean_10_lat','tf_mean_10_lng','tf_prev_lat','tf_prev_lng','tf_mean_2_lat_f',
'tf_mean_2_lng_f','tf_mean_4_lat_f','tf_mean_4_lng_f','tf_mean_6_lat_f','tf_mean_6_lng_f',
'tf_mean_8_lat_f','tf_mean_8_lng_f','tf_mean_10_lat_f','tf_mean_10_lng_f',
'tf_future_lat','tf_future_lng','tf_mobile_lat','tf_mobile_lng', 'dist_diff']
raw_mapmatch = pd.read_csv(filename_data_train_mapmatch,sep='\t',names=columns)
print "raw_mapmatch.shape = " + str(raw_mapmatch.shape)
raw_mapmatch

/Users/ngyibin/Documents/DataSparkPrograms/venv/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


raw_mapmatch.shape = (3812138, 75)


,imsi,cellid,lbs_timestamp,event_type,network,antenna,lbs_lat,lbs_lng,radius,azimuth,...,tf_mean_6_lng_f,tf_mean_8_lat_f,tf_mean_8_lng_f,tf_mean_10_lat_f,tf_mean_10_lng_f,tf_future_lat,tf_future_lng,tf_mobile_lat,tf_mobile_lng,dist_diff
0,5250165E2F5EB4FED08A4F9BC8706C602040EA,525-1-377-37909,2016-11-16 06:44:47,105,3G,33,1.303917,103.861639,100,270,...,NaN,NaN,NaN,NaN,NaN,13424.229251,32713.678287,11885.442422,29600.385857,2267.634662
1,525016694AEC317ECE1A47542D46AA86138509,525-1-707-7309532,2016-11-16 07:01:45,210,LTE,25,1.297500,103.881000,200,220,...,30517.135551,13612.853618,30327.005533,13612.853618,30327.005533,12904.739190,30568.146044,12819.905597,30839.181796,2403.876550
2,525016694AEC317ECE1A47542D46AA86138509,525-1-707-7309532,2016-11-16 07:01:37,210,LTE,25,1.297500,103.881000,200,220,...,30517.135551,13612.853618,30327.005533,13612.853618,30327.005533,12883.093770,33242.332485,12819.905597,30839.181796,2403.876550
3,525016694AEC317ECE1A47542D46AA86138509,525-1-707-7309532,2016-11-16 07:01:11,210,LTE,25,1.297500,103.881000,200,220,...,30783.317578,13612.853618,30327.005533,13612.853618,30327.005533,12883.093770,33242.332485,12819.905597,30839.181796,2403.876550
4,525016694AEC317ECE1A47542D46AA86138509,525-1-707-7309532,2016-11-16 07:01:08,210,LTE,25,1.297500,103.881000,200,220,...,30783.317578,13612.853618,30327.005533,13612.853618,30327.005533,12883.093770,33242.332485,12819.905597,30839.181796,2403.876550
5,525016694AEC317ECE1A47542D46AA86138509,525-1-707-7474732,2016-11-16 07:00:23,203,LTE,65,1.297083,103.887528,200,230,...,30851.949877,13612.853618,30327.005533,13612.853618,30327.005533,12883.093770,33242.332485,12819.905597,30839.181796,3129.572712
6,525016694AEC317ECE1A47542D46AA86138509,525-1-707-7312032,2016-11-16 06:58:34,203,LTE,48,1.302944,103.907722,200,210,...,30450.667334,13570.181220,30850.713259,13612.853618,30327.005533,12836.710729,33968.845564,12819.905597,30839.181796,5418.467331
7,525016694AEC317ECE1A47542D46AA86138509,525-1-707-7312132,2016-11-16 06:58:17,200,LTE,59,1.302667,103.912250,200,250,...,30568.146044,13490.402389,30851.949877,13612.853618,30327.005533,13489.165508,36216.398792,12819.905597,30839.181796,5915.419085
8,525016694AEC317ECE1A47542D46AA86138509,525-1-707-7464231,2016-11-16 06:55:58,200,LTE,10,1.308944,103.935278,200,60,...,33387.635101,12904.739190,30568.146044,13463.654835,30793.055944,13458.243481,36720.320630,12412.994598,31273.396095,8197.145153
9,525016694AEC317ECE1A47542D46AA86138509,525-1-707-7393131,2016-11-16 06:55:22,200,LTE,7,1.310250,103.940028,200,70,...,33484.503511,12883.093770,33242.332485,13188.191108,30521.772869,14157.081301,39283.211449,12412.994598,31273.396095,8744.641317


## Common functions

In [2]:
num_partitions = 10 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

## Remove records with unmatched LBS records

In [3]:
raw_data_mapmatch = raw_mapmatch[(raw_mapmatch['dist_diff'].notnull()) & (raw_mapmatch['mobile_zone'].notnull())]
print "raw_data_mapmatch.shape = " + str(raw_data_mapmatch.shape)

raw_data_mapmatch.shape = (3812138, 75)


## Convert time columns to datetime

In [4]:
raw_data_mapmatch.loc[:, 'mobile_timestamp'] = pd.to_datetime(raw_data_mapmatch['mobile_timestamp'],format='%Y-%m-%d %H:%M:%S')
raw_data_mapmatch.loc[:, 'lbs_timestamp'] = pd.to_datetime(raw_data_mapmatch['lbs_timestamp'],format='%Y-%m-%d %H:%M:%S')
# raw_data_test_mapmatch.loc[:, 'mobile_timestamp'] = pd.to_datetime(raw_data_test_mapmatch['mobile_timestamp'],format='%Y-%m-%d %H:%M:%S')
# raw_data_test_mapmatch.loc[:, 'lbs_timestamp'] = pd.to_datetime(raw_data_test_mapmatch['lbs_timestamp'],format='%Y-%m-%d %H:%M:%S')

## Keep only relevant rows with time difference >= timeDiffMin and <= timeDiffMax seconds

In [5]:
relevant_mapmatch = raw_data_mapmatch[((raw_data_mapmatch['mobile_timestamp'] - raw_data_mapmatch['lbs_timestamp']).dt.total_seconds() >= timeDiffMin) & 
                    ((raw_data_mapmatch['mobile_timestamp'] - raw_data_mapmatch['lbs_timestamp']).dt.total_seconds() <= timeDiffMax)]
# relevant_test_mapmatch = raw_data_test_mapmatch[((raw_data_test_mapmatch['mobile_timestamp'] - raw_data_test_mapmatch['lbs_timestamp']).dt.total_seconds() >= timeDiffMin) & 
#                     ((raw_data_test_mapmatch['mobile_timestamp'] - raw_data_test_mapmatch['lbs_timestamp']).dt.total_seconds() <= timeDiffMax)]
print "relevant_mapmatch.shape = " + str(relevant_mapmatch.shape)
# print "relevant_test_mapmatch.shape = " + str(relevant_test_mapmatch.shape)

relevant_mapmatch.shape = (980079, 75)


## Add in hour

In [6]:
def myround(x, base=15):
    return int(base * round(float(x)/base))

relevant_mapmatch.loc[:, 'day'] = relevant_mapmatch['lbs_timestamp'].dt.weekday
relevant_mapmatch.loc[:, 'hour'] = relevant_mapmatch['lbs_timestamp'].dt.hour
relevant_mapmatch.loc[:, 'minute'] = relevant_mapmatch['lbs_timestamp'].dt.minute
relevant_mapmatch.loc[:, '5min'] = relevant_mapmatch['minute'].map(lambda x :myround(x,5))
relevant_mapmatch.loc[:, '10min'] = relevant_mapmatch['minute'].map(lambda x :myround(x,10))
relevant_mapmatch[['lbs_timestamp', 'day', 'hour', 'minute', '5min', '10min']]

/Users/ngyibin/Documents/DataSparkPrograms/venv/lib/python2.7/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/ngyibin/Documents/DataSparkPrograms/venv/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,lbs_timestamp,day,hour,minute,5min,10min
8,2016-11-16 06:55:58,2,6,55,55,60
9,2016-11-16 06:55:22,2,6,55,55,60
15,2016-11-16 18:18:34,2,18,18,20,20
16,2016-11-16 20:15:31,2,20,15,15,20
18,2016-11-16 20:11:47,2,20,11,10,10
21,2016-11-16 20:08:49,2,20,8,10,10
22,2016-11-16 20:07:18,2,20,7,5,10
27,2016-11-16 19:20:28,2,19,20,20,20
32,2016-11-16 18:06:16,2,18,6,5,10
38,2016-11-16 10:03:32,2,10,3,5,0


## Retain data for only 1 day

In [7]:
date = "2016-11-16"
relevant_mapmatch = relevant_mapmatch[relevant_mapmatch['lbs_timestamp'].dt.strftime("%Y-%m-%d") == date]
# relevant_mapmatch = relevant_mapmatch[relevant_mapmatch['lbs_timestamp'] ]
relevant_mapmatch

,imsi,cellid,lbs_timestamp,event_type,network,antenna,lbs_lat,lbs_lng,radius,azimuth,...,tf_future_lat,tf_future_lng,tf_mobile_lat,tf_mobile_lng,dist_diff,day,hour,minute,5min,10min
8,525016694AEC317ECE1A47542D46AA86138509,525-1-707-7464231,2016-11-16 06:55:58,200,LTE,10,1.308944,103.935278,200,60,...,13458.243481,36720.320630,12412.994598,31273.396095,8197.145153,2,6,55,55,60
9,525016694AEC317ECE1A47542D46AA86138509,525-1-707-7393131,2016-11-16 06:55:22,200,LTE,7,1.310250,103.940028,200,70,...,14157.081301,39283.211449,12412.994598,31273.396095,8744.641317,2,6,55,55,60
15,525016BD62DB0827C58FB47F1CE3DEA858D7D4,525-1-307-59541,2016-11-16 18:18:34,210,3G,NaN,1.286278,103.878722,50,0,...,11030.864326,31789.306327,10183.637510,28887.067739,4350.434597,2,18,18,20,20
16,525016E50FB8B81210D4678BE69050A4CF07D8,525-1-738-7388636,2016-11-16 20:15:31,200,LTE,18,1.267556,103.807167,200,"310,40",...,9939.316757,26527.496704,10209.804224,28893.936905,3924.713216,2,20,15,15,20
18,525016E50FB8B81210D4678BE69050A4CF07D8,525-1-738-7442231,2016-11-16 20:11:47,210,LTE,28,1.268750,103.825889,200,80,...,9682.663930,27108.707168,10456.338622,28997.684456,2041.216897,2,20,11,10,10
21,525016E50FB8B81210D4678BE69050A4CF07D8,525-1-741-7377333,2016-11-16 20:08:49,200,LTE,39,1.273917,103.844139,200,300,...,10378.409547,28985.274995,10121.184043,29113.489776,139.149484,2,20,8,10,10
22,525016E50FB8B81210D4678BE69050A4CF07D8,525-1-709-7439832,2016-11-16 20:07:18,210,LTE,18,1.274972,103.843556,200,270,...,10257.813640,29139.852246,10121.184043,29113.489776,257.041833,2,20,7,5,10
27,525016E50FB8B81210D4678BE69050A4CF07D8,525-1-709-7439832,2016-11-16 19:20:28,203,LTE,18,1.274972,103.843556,200,270,...,10375.317344,29074.929800,10418.261856,29292.896090,222.149079,2,19,20,20,20
32,525016E50FB8B81210D4678BE69050A4CF07D8,525-1-709-7439833,2016-11-16 18:06:16,200,LTE,18,1.274972,103.843556,200,350,...,10727.828457,29254.239411,10158.110881,29024.891784,222.895218,2,18,6,5,10
38,525016D132267935AC72AB35E086AA2B89E860,525-1-741-7422434,2016-11-16 10:03:32,210,LTE,20,1.272778,103.840000,200,20,...,10131.033328,28679.212038,10260.411090,29235.690141,571.301384,2,10,3,5,0


## Impute 2 mins average location

In [8]:
'''
    'tf_mean_2_lat','tf_mean_2_lng',
    'tf_mean_4_lat','tf_mean_4_lng',
    'tf_mean_6_lat','tf_mean_6_lng',
    'tf_mean_8_lat','tf_mean_8_lng',
    'tf_mean_10_lat','tf_mean_10_lng',

    'tf_mean_2_lat_f','tf_mean_2_lng_f',
    'tf_mean_4_lat_f','tf_mean_4_lng_f',
    'tf_mean_6_lat_f','tf_mean_6_lng_f',
    'tf_mean_8_lat_f','tf_mean_8_lng_f',
    'tf_mean_10_lat_f','tf_mean_10_lng_f',
'''

def _fill_average_location(row):
    if pd.isnull(row['tf_mean_4_lat']):
        row['tf_mean_4_lat'] = row['tf_mean_2_lat']
    if pd.isnull(row['tf_mean_6_lat']):
        row['tf_mean_6_lat'] = row['tf_mean_4_lat']
    if pd.isnull(row['tf_mean_8_lat']):
        row['tf_mean_8_lat'] = row['tf_mean_6_lat']
    if pd.isnull(row['tf_mean_10_lat']):
        row['tf_mean_10_lat'] = row['tf_mean_8_lat']
        
    if pd.isnull(row['tf_mean_4_lng']):
        row['tf_mean_4_lng'] = row['tf_mean_2_lng']
    if pd.isnull(row['tf_mean_6_lng']):
        row['tf_mean_6_lng'] = row['tf_mean_4_lng']
    if pd.isnull(row['tf_mean_8_lng']):
        row['tf_mean_8_lng'] = row['tf_mean_6_lng']
    if pd.isnull(row['tf_mean_10_lng']):
        row['tf_mean_10_lng'] = row['tf_mean_8_lng']
        
    if pd.isnull(row['tf_mean_4_lat_f']):
        row['tf_mean_4_lat_f'] = row['tf_mean_2_lat_f']
    if pd.isnull(row['tf_mean_6_lat_f']):
        row['tf_mean_6_lat_f'] = row['tf_mean_4_lat_f']
    if pd.isnull(row['tf_mean_8_lat_f']):
        row['tf_mean_8_lat_f'] = row['tf_mean_6_lat_f']
    if pd.isnull(row['tf_mean_10_lat_f']):
        row['tf_mean_10_lat_f'] = row['tf_mean_8_lat_f']
        
    if pd.isnull(row['tf_mean_4_lng_f']):
        row['tf_mean_4_lng_f'] = row['tf_mean_2_lng_f']
    if pd.isnull(row['tf_mean_6_lng_f']):
        row['tf_mean_6_lng_f'] = row['tf_mean_4_lng_f']
    if pd.isnull(row['tf_mean_8_lng_f']):
        row['tf_mean_8_lng_f'] = row['tf_mean_6_lng_f']
    if pd.isnull(row['tf_mean_10_lng_f']):
        row['tf_mean_10_lng_f'] = row['tf_mean_8_lng_f']
    
    return row

def fill_average_location(df):
    df = df.apply(_fill_average_location, axis=1)
    return df

# time1 = time.time()
# relevant_mapmatch = fill_average_location(relevant_mapmatch)
# # relevant_test_mapmatch = fill_average_location(relevant_test_mapmatch)
# print 'function took %0.3f s' % (time.time()-time1)
# # function took 42.502 s

time1 = time.time()
relevant_mapmatch = parallelize_dataframe(relevant_mapmatch, fill_average_location)
# relevant_test_mapmatch = fill_average_location(relevant_test_mapmatch)
print 'function took %0.3f s' % (time.time()-time1)
# function took 21.306 s for num_partitions = 10, num_cores = 4


function took 14.795 s


## Convert Categorical Variables

In [9]:
columns_to_be_labeled = [
    'cellid',
    'lbs_zone',
    'prev_cell_id',
    'future_cell_id',
    'network',
    'day',
    'hour',
    '5min',
    '10min'
]

# print LabelEncoder().fit_transform(relevant_mapmatch[['cellid']])
# # [1850 1804  120 ..., 2058 2364 2364]

# print LabelEncoder().fit(relevant_mapmatch[['cellid']])
# # LabelEncoder()

# print LabelEncoder().transform(relevant_mapmatch[['cellid']])
# # NotFittedError: This LabelEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

for column in columns_to_be_labeled:
    relevant_mapmatch.loc[:, column] = LabelEncoder().fit_transform(relevant_mapmatch[[column]])
#     relevant_test_mapmatch.loc[:, column] = LabelEncoder().fit_transform(relevant_test_mapmatch[[column]])
relevant_mapmatch.head()

/Users/ngyibin/Documents/DataSparkPrograms/venv/lib/python2.7/site-packages/sklearn/preprocessing/label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,imsi,cellid,lbs_timestamp,event_type,network,antenna,lbs_lat,lbs_lng,radius,azimuth,...,tf_future_lat,tf_future_lng,tf_mobile_lat,tf_mobile_lng,dist_diff,day,hour,minute,5min,10min
8,525016694AEC317ECE1A47542D46AA86138509,1850,2016-11-16 06:55:58,200,2,10,1.308944,103.935278,200,60,...,13458.243481,36720.320630,12412.994598,31273.396095,8197.145153,0,6,55,11,6
9,525016694AEC317ECE1A47542D46AA86138509,1804,2016-11-16 06:55:22,200,2,7,1.310250,103.940028,200,70,...,14157.081301,39283.211449,12412.994598,31273.396095,8744.641317,0,6,55,11,6
15,525016BD62DB0827C58FB47F1CE3DEA858D7D4,120,2016-11-16 18:18:34,210,1,NaN,1.286278,103.878722,50,0,...,11030.864326,31789.306327,10183.637510,28887.067739,4350.434597,0,18,18,4,2
16,525016E50FB8B81210D4678BE69050A4CF07D8,2931,2016-11-16 20:15:31,200,2,18,1.267556,103.807167,200,"310,40",...,9939.316757,26527.496704,10209.804224,28893.936905,3924.713216,0,20,15,3,2
18,525016E50FB8B81210D4678BE69050A4CF07D8,2967,2016-11-16 20:11:47,210,2,28,1.268750,103.825889,200,80,...,9682.663930,27108.707168,10456.338622,28997.684456,2041.216897,0,20,11,2,1


## Split into train and test set

In [10]:
# features = ['cellid','event_type','network','antenna','lbs_lat','lbs_lng',
# 'radius','mean_2_lat','mean_2_lng','mean_4_lat','mean_4_lng','mean_6_lat',
# 'mean_6_lng','mean_8_lat','mean_8_lng','mean_10_lat','mean_10_lng','prev_cell_id',
# 'prev_lat', 'prev_lng','mean_2_lat_f','mean_2_lng_f','mean_4_lat_f','mean_4_lng_f',
# 'mean_6_lat_f','mean_6_lng_f','mean_8_lat_f','mean_8_lng_f','mean_10_lat_f','mean_10_lng_f',
# 'future_cell_id','future_lat','future_lng', 'lbs_zone','tf_lbs_lat','tf_lbs_lng','tf_mean_2_lat',
# 'tf_mean_2_lng','tf_mean_4_lat','tf_mean_4_lng','tf_mean_6_lat','tf_mean_6_lng','tf_mean_8_lat',
# 'tf_mean_8_lng','tf_mean_10_lat','tf_mean_10_lng','tf_prev_lat','tf_prev_lng','tf_mean_2_lat_f',
# 'tf_mean_2_lng_f','tf_mean_4_lat_f','tf_mean_4_lng_f','tf_mean_6_lat_f','tf_mean_6_lng_f',
# 'tf_mean_8_lat_f','tf_mean_8_lng_f','tf_mean_10_lat_f','tf_mean_10_lng_f',
# 'tf_future_lat','tf_future_lng', 'dist_diff']
features = [
    'cellid',
#     'lbs_zone_e',
#     'prev_cell_id',
#     'future_cell_id',
#     'ratio',
    
#     'tf_lbs_lat','tf_lbs_lng',
    
    'tf_mean_2_lat','tf_mean_2_lng',
    'tf_mean_4_lat','tf_mean_4_lng',
    'tf_mean_6_lat','tf_mean_6_lng',
    'tf_mean_8_lat','tf_mean_8_lng',
    'tf_mean_10_lat','tf_mean_10_lng',

    'tf_mean_2_lat_f','tf_mean_2_lng_f',
    'tf_mean_4_lat_f','tf_mean_4_lng_f',
    'tf_mean_6_lat_f','tf_mean_6_lng_f',
    'tf_mean_8_lat_f','tf_mean_8_lng_f',
    'tf_mean_10_lat_f','tf_mean_10_lng_f',
    
#     'tf_prev_lat','tf_prev_lng',
#     'tf_future_lat','tf_future_lng',
    
    'hour',
    '5min',
    '10min',
    'day',
#             'network', 
#             'cell_type',
#             'location_type',
#             'antenna_height',
]
# X_train = relevant_mapmatch[features]
# # X_sample = relevant_test_mapmatch[features]
# # y_train = relevant_mapmatch[['mobile_zone', 'lbs_zone']]
# # y_sample = relevant_test_mapmatch[['mobile_zone', 'lbs_zone']]
# y_train = pd.factorize(relevant_mapmatch['mobile_zone'])[0]
# # y_sample = pd.factorize(relevant_test_mapmatch['mobile_zone'])[0]
# print "X_train.shape = " + str(X_train.shape)
# print "y_train.shape = " + str(y_train.shape)
# # print "X_sample.shape = " + str(X_sample.shape)
# # print "y_sample.shape = " + str(y_sample.shape)
# print "X_train = " + str(X_train)
# print "y_train = " + str(y_train)

train, test = train_test_split(relevant_mapmatch, test_size = 0.3)
X_train = train[features]
X_sample = test[features]
y_train, mobile_zone_uniques = pd.factorize(train['mobile_zone'])
y_sample = test['mobile_zone']
print "X_train.shape = " + str(X_train.shape)
print "y_train.shape = " + str(y_train.shape)
print "X_sample.shape = " + str(X_sample.shape)
print "y_sample.shape = " + str(y_sample.shape)
print "X_train = " + str(X_train)
print "y_train = " + str(y_train)
print "mobile_zone_uniques = " + str(mobile_zone_uniques)

X_train.shape = (63170, 25)
y_train.shape = (63170,)
X_sample.shape = (27073, 25)
y_sample.shape = (27073,)
X_train =         cellid  tf_mean_2_lat  tf_mean_2_lng  tf_mean_4_lat  tf_mean_4_lng  \
78203     2877   10124.848922   25329.213855   11120.538206   24622.074457   
68085     2078   11626.422575   29508.364412   11782.269593   29532.478463   
279768    3313   10511.374265   29680.872624   11092.708381   30024.034121   
333724    2330   11272.056140   30778.371106   11272.056140   30778.371106   
38428     2529   12400.710142   30244.564332   12400.710142   30244.564332   
94249     2062   11264.841000   29418.091297   11802.884278   29526.638878   
31028     2290   11340.084601   30017.851031   11388.786794   30059.586889   
269760    3129   12904.739190   30568.146044   12904.739190   30568.146044   
250704    1717   14002.471164   31350.306934   13133.562193   30976.229987   
217009    3312   10891.715203   30229.621865   11272.056140   30778.371106   
25664     3327   10066.0

## Train Random Forest Classifier

In [11]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features='sqrt', 
                               criterion='entropy',
#                                min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)





RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=5, oob_score=False, random_state=20,
            verbose=0, warm_start=False)

## Apply Classifier to Test Data

In [12]:
est_sz = model.predict(X_sample)
est_sz[:10]

array([ 4,  1,  8,  7,  0,  7, 14,  8,  0, 13])

In [13]:
# View the predicted probabilities of the first 10 observations
model.predict_proba(X_sample)[0:10]
# There are three species of plant, thus [ 1. , 0. , 0. ] tells us that the classifier is certain that 
# the plant is the first class. Taking another example, [ 0.9, 0.1, 0. ] tells us that the classifier gives 
# a 90% probability the plant belongs to the first class and a 10% probability the plant belongs to the 
# second class. Because 90 is greater than 10, the classifier predicts the plant is the first class.

array([[ 0.09      ,  0.038     ,  0.018     ,  0.01      ,  0.502     ,
         0.01      ,  0.004     ,  0.168     ,  0.008     ,  0.082     ,
         0.006     ,  0.        ,  0.048     ,  0.002     ,  0.01      ,
         0.004     ],
       [ 0.012     ,  0.818     ,  0.006     ,  0.016     ,  0.026     ,
         0.032     ,  0.        ,  0.058     ,  0.        ,  0.016     ,
         0.002     ,  0.        ,  0.006     ,  0.002     ,  0.        ,
         0.006     ],
       [ 0.06      ,  0.02      ,  0.006     ,  0.014     ,  0.004     ,
         0.048     ,  0.018     ,  0.026     ,  0.264     ,  0.        ,
         0.252     ,  0.06      ,  0.002     ,  0.062     ,  0.16      ,
         0.004     ],
       [ 0.        ,  0.        ,  0.002     ,  0.136     ,  0.034     ,
         0.004     ,  0.006     ,  0.65      ,  0.03      ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.002     ,
         0.136     ],
       [ 1.        ,  0.        ,  0

## Evaluate Classifier

In [14]:
# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]
est_szName

Index([u'CITY HALL', u'CHINA SQUARE', u'CENTRAL SUBZONE', u'MARINA CENTRE',
       u'CHINATOWN', u'MARINA CENTRE', u'CECIL', u'CENTRAL SUBZONE',
       u'CHINATOWN', u'ANSON',
       ...
       u'CHINATOWN', u'CHINA SQUARE', u'MARINA CENTRE', u'RAFFLES PLACE',
       u'CITY HALL', u'CHINATOWN', u'CHINA SQUARE', u'RAFFLES PLACE',
       u'RAFFLES PLACE', u'BOAT QUAY'],
      dtype='object', length=27073)

In [15]:
# View the ACTUAL species for the first five observations
y_sample.head()

77575         CITY HALL
165754     CHINA SQUARE
18277     TANJONG PAGAR
326500    MARINA CENTRE
225317        CHINATOWN
Name: mobile_zone, dtype: object

In [16]:
# Create confusion matrix
pd.crosstab(y_sample, est_szName, rownames=['Actual Subzones'], colnames=['Predicted Subzones'])

Predicted Subzones,ANSON,BAYFRONT SUBZONE,BOAT QUAY,CECIL,CENTRAL SUBZONE,CHINA SQUARE,CHINATOWN,CITY HALL,CLIFFORD PIER,MARINA CENTRE,MARINA SOUTH,MAXWELL,PHILLIP,RAFFLES PLACE,STRAITS VIEW,TANJONG PAGAR
Actual Subzones,,,,,,,,,,,,,,,,
ANSON,1081,4,0,20,81,0,112,8,1,1,1,0,0,3,13,49
BAYFRONT SUBZONE,4,933,1,3,88,0,3,29,6,66,27,0,0,23,6,1
BOAT QUAY,1,1,972,3,3,86,31,101,0,6,0,0,28,62,0,3
CECIL,21,2,0,1331,194,29,110,10,5,13,4,3,1,93,2,76
CENTRAL SUBZONE,45,58,1,154,1891,13,46,23,28,34,13,1,1,99,40,38
CHINA SQUARE,2,1,58,32,11,1251,195,34,0,7,0,0,18,134,0,4
CHINATOWN,132,3,14,105,34,133,3223,28,2,7,0,30,0,66,1,72
CITY HALL,7,7,88,12,24,16,24,2932,16,172,1,1,1,64,2,2
CLIFFORD PIER,1,37,5,10,62,4,4,41,387,46,0,0,0,150,1,0


In [17]:
# View a list of the features and their importance scores

# print clf.feature_importances_
# [ 0.04840973  0.08164525  0.07227249  0.05280516  0.05267236  0.05045209
#   0.04981372  0.04605795  0.04400915  0.04052635  0.03990643  0.07691532
#   0.06918858  0.04139624  0.04240057  0.03232955  0.03340456  0.03181781
#   0.03010951  0.03221942  0.03164774]

imp = list(zip(train[features], model.feature_importances_))
imp.sort(key=lambda tup: tup[1]) 
imp

[('day', 0.0),
 ('10min', 0.018639406780648841),
 ('5min', 0.026187593204601536),
 ('tf_mean_8_lng_f', 0.02788707771718384),
 ('tf_mean_10_lng_f', 0.028129397724622026),
 ('tf_mean_10_lat_f', 0.028282552752871289),
 ('tf_mean_8_lat_f', 0.028940095400212359),
 ('tf_mean_6_lat_f', 0.029597239238477509),
 ('tf_mean_6_lng_f', 0.030717278595191404),
 ('tf_mean_10_lat', 0.035358184051850747),
 ('tf_mean_4_lat_f', 0.036439365217453244),
 ('tf_mean_10_lng', 0.036728555357964973),
 ('hour', 0.037418568909503208),
 ('tf_mean_4_lng_f', 0.038733135693229934),
 ('tf_mean_8_lng', 0.039870627097672647),
 ('cellid', 0.040753836739741353),
 ('tf_mean_8_lat', 0.041428602557307057),
 ('tf_mean_6_lng', 0.044164738457341181),
 ('tf_mean_4_lng', 0.047664237421688481),
 ('tf_mean_6_lat', 0.047797565727652404),
 ('tf_mean_4_lat', 0.054737019370860431),
 ('tf_mean_2_lng_f', 0.065019234338539442),
 ('tf_mean_2_lng', 0.068146254753003163),
 ('tf_mean_2_lat_f', 0.073587817050010565),
 ('tf_mean_2_lat', 0.07377161

## Compute accuracy

In [18]:
# print classification_report(data_test[target], output)
# print accuracy_score(data_test[target], output)
print classification_report(y_sample, est_szName)
print accuracy_score(y_sample, est_szName)

                  precision    recall  f1-score   support

           ANSON       0.77      0.79      0.78      1374
BAYFRONT SUBZONE       0.80      0.78      0.79      1190
       BOAT QUAY       0.75      0.75      0.75      1297
           CECIL       0.68      0.70      0.69      1894
 CENTRAL SUBZONE       0.70      0.76      0.73      2485
    CHINA SQUARE       0.73      0.72      0.72      1747
       CHINATOWN       0.79      0.84      0.81      3850
       CITY HALL       0.83      0.87      0.85      3369
   CLIFFORD PIER       0.76      0.52      0.62       748
   MARINA CENTRE       0.85      0.86      0.85      2857
    MARINA SOUTH       0.86      0.74      0.80       618
         MAXWELL       0.80      0.48      0.60       361
         PHILLIP       0.76      0.48      0.59       518
   RAFFLES PLACE       0.73      0.79      0.76      2791
    STRAITS VIEW       0.85      0.75      0.79       673
   TANJONG PAGAR       0.75      0.65      0.70      1301

     avg / t

## Tuning the Random Forest - n_estimators

In [25]:
model = RandomForestClassifier(n_estimators=5, 
                               max_features='sqrt', 
                               criterion='entropy',
#                                min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

(63170, 25)
(63170,)
0.692571935138


In [20]:
model = RandomForestClassifier(n_estimators=50, 
                               max_features='sqrt', 
                               criterion='entropy',
#                                min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

0.761792191482


In [21]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features='sqrt', 
                               criterion='entropy',
#                                min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

0.77128504414


## Tuning the Random Forest - max_features

In [22]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features='sqrt', 
                               criterion='entropy',
#                                min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

0.77128504414


In [23]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features='log2', 
                               criterion='entropy',
#                                min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

0.771432792819


In [24]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features=0.2, 
                               criterion='entropy',
#                                min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

KeyboardInterrupt: 

In [ ]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features=None, 
                               criterion='entropy',
#                                min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

## Tuning the Random Forest - criterion

In [ ]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features='log2', 
                               criterion='entropy',
#                                min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

In [ ]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features='log2', 
                               criterion='gini',
#                                min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

## Tuning the Random Forest - min_samples_leaf

In [ ]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features='log2', 
                               criterion='gini',
                               min_samples_leaf=1,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

In [ ]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features='log2', 
                               criterion='gini',
                               min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

In [ ]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features='log2', 
                               criterion='gini',
                               min_samples_leaf=50,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

## Tuning the Random Forest - oob_score

In [ ]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features='log2', 
                               criterion='gini',
                               oob_score = False,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

In [ ]:
model = RandomForestClassifier(n_estimators=500, 
                               max_features='log2', 
                               criterion='gini',
                               oob_score = True,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

est_sz = model.predict(X_sample)

# For each predicted class, replace with subzone name
est_szName = mobile_zone_uniques[est_sz]

print accuracy_score(y_sample, est_szName)

## Random Forest Complete Example

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

import pandas as pd

# Load dataframe
columns = ['imsi','cellid','lbs_timestamp','event_type','network','antenna','lbs_lat','lbs_lng',
'radius','azimuth','mean_2_lat','mean_2_lng','mean_4_lat','mean_4_lng','mean_6_lat',
'mean_6_lng','mean_8_lat','mean_8_lng','mean_10_lat','mean_10_lng','prev_cell_id',
'prev_lat', 'prev_lng', 'prev_timestamp','mean_2_lat_f','mean_2_lng_f','mean_4_lat_f','mean_4_lng_f',
'mean_6_lat_f','mean_6_lng_f','mean_8_lat_f','mean_8_lng_f','mean_10_lat_f','mean_10_lng_f',
'future_cell_id','future_lat','future_lng', 'future_timestamp', 'mobile_timestamp','host','url','useragent',
'mobile_lat','mobile_lng','mobile_zone','lbs_zone','tf_lbs_lat','tf_lbs_lng','tf_mean_2_lat',
'tf_mean_2_lng','tf_mean_4_lat','tf_mean_4_lng','tf_mean_6_lat','tf_mean_6_lng','tf_mean_8_lat',
'tf_mean_8_lng','tf_mean_10_lat','tf_mean_10_lng','tf_prev_lat','tf_prev_lng','tf_mean_2_lat_f',
'tf_mean_2_lng_f','tf_mean_4_lat_f','tf_mean_4_lng_f','tf_mean_6_lat_f','tf_mean_6_lng_f',
'tf_mean_8_lat_f','tf_mean_8_lng_f','tf_mean_10_lat_f','tf_mean_10_lng_f',
'tf_future_lat','tf_future_lng','tf_mobile_lat','tf_mobile_lng', 'dist_diff']
raw_mapmatch = pd.read_csv(filename_data_train_mapmatch,sep='\t',names=columns)

# Convert categorical variable
columns_to_be_labeled = [
    'cellid',
    'lbs_zone',
    'prev_cell_id',
    'future_cell_id',
    'network'
]
for column in columns_to_be_labeled:
    relevant_mapmatch.loc[:, column] = LabelEncoder().fit_transform(relevant_mapmatch[[column]])

# Select features and generate training and test set
features = [
    'cellid',
    'tf_mean_2_lat','tf_mean_2_lng',
    'tf_mean_4_lat','tf_mean_4_lng',
    'tf_mean_6_lat','tf_mean_6_lng',
    'tf_mean_8_lat','tf_mean_8_lng',
    'tf_mean_10_lat','tf_mean_10_lng',

    'tf_mean_2_lat_f','tf_mean_2_lng_f',
    'tf_mean_4_lat_f','tf_mean_4_lng_f',
    'tf_mean_6_lat_f','tf_mean_6_lng_f',
    'tf_mean_8_lat_f','tf_mean_8_lng_f',
    'tf_mean_10_lat_f','tf_mean_10_lng_f',
]
train, test = train_test_split(relevant_mapmatch, test_size = 0.3)
X_train = train[features]
X_sample = test[features]
y_train, mobile_zone_uniques = pd.factorize(train['mobile_zone'])
y_sample = test['mobile_zone']

# Train RF Classifier
model = RandomForestClassifier(n_estimators=500, 
                               max_features='sqrt', 
                               criterion='entropy',
#                                min_samples_leaf=10,
                               n_jobs=5,
                              random_state=20)
model.fit(X_train, y_train)

# Apply Classifier to test data
est_sz = model.predict(X_sample)
est_szName = mobile_zone_uniques[est_sz]

# Compute accuracy
print accuracy_score(y_sample, est_szName)